In [1]:
%%capture --no-display
from dask.distributed import Client
client = Client(address = 'tcp://192.168.62.207:8686')
client

Client Scheduler: tcp://192.168.62.207:8686 Dashboard: http://192.168.62.207:8687/status,Cluster Workers: 7 Cores: 56 Memory: 471.41 GB


In [11]:
import dask.bag as db
from takco.util import robust_json_loads_lines
fnames = '/export/scratch1/home/kruit/scratch/output/tabel-small-4/1-cluster/*'
bag = db.read_text(fnames).map_partitions(robust_json_loads_lines).persist()
bag.count().compute()

19928

In [18]:
from collections import Counter
hid_freq = Counter(dict(bag.map(lambda t: (t.headerId, len(t.provenance.get('concat', [0])))).compute()))
len(hid_freq)

19928

In [50]:
import pandas as pd
cells = bag.map(lambda t: [(t.headerId, c.lower()) for r in t.head for c in r if c[0]!='_']).flatten()

hcells = pd.DataFrame(cells.compute(), columns=['hid', 'header'])
display.display(hcells.head())

hfreq = pd.DataFrame(hid_freq.items(), columns=['hid', 'freq'])
display.display(hfreq.head())

,hid,header
0,63588625754282,team
1,63588625754282,win
2,63588625754282,loss
3,63588625754282,sets won
4,63588625754282,sets lost


,hid,freq
0,63588625754282,2
1,98304191789482,1
2,207795389965572,1
3,1842756628326155,1
4,2606468435400551,1


In [68]:
import duckdb
con = duckdb.connect(database=':memory:', read_only=False)
con.register('cell', hcells)
con.register('freq', hfreq)
con.execute("""
SELECT c2.header AS cell, SUM(f.freq) AS freq
FROM cell c1 JOIN cell c2 ON c1.hid=c2.hid JOIN freq f ON f.hid=c1.hid
WHERE c1.header=? AND c2.header != c1.header
GROUP BY cell
ORDER BY freq DESC
""", ['%']).fetchdf().head(20)

,cell,freq
0,party,3502.0
1,votes,1919.0
2,,1848.0
3,candidate,1739.0
4,± pp,1521.0
5,expenditures,509.0
6,number,173.0
7,seats,148.0
8,republican,132.0
9,democrat,132.0
